In [1]:
import pandas as pd
import datetime as dt
pd.set_option('display.max_columns', None)

# Con Edison

## Projects

In [2]:
coned_df = pd.read_csv('data/csv/coned_2020-02.csv', skiprows=2)
coned_df.head(3)

/Applications/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (41,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Company,Developer,Application / Job #,Division,City/Town,Zip Code,Circuit ID,Substation,Hybrid\n(Y/N),Related Application/Job #,PV\n(kWAC),ESS\n(kWAC),WIND\n(kWAC),MT\n(kWAC),SG\n(kWAC),IG\n(kWAC),FW\n(kWAC),FC\n(kWAC),CHP\n(kWAC),GT\n(kWAC),HYDRO\n(kWAC),ICE\n(kWAC),ST\n(kWAC),OTHER\n(kWAC),Metering\n(NA / NM / RNM / CDG),Value Stack\n(Y/N),Start Date,End Date,Calculated Duration,Application Approved Date \n(Utility),Start Date (Must Match Application Approved Date),End Date.1,Calculated Duration.1,Payment Received Date,Start Date.1,End Date.2,Calculated Duration.2,CESIR cost to customer,CESIR cost to utility,Estimated Costs by Utility,Actual Customer Project Costs,Actual Utility Project Costs,Down Payment Date,Full Payment Date,Construction Start Date,Construction Complete Date,Verification Testing or Final Acceptance Date,Final Letter of Acceptance Date,Project Complete (Y/N),Utility Retention of REC (Y/N)
0,CECONY,"Sunrun Installation Services, Inc.",MC-473671,CENY-Q,Queens,11436.0,9344,Jamaica,N,NaN,2.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NM,N,2/29/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,N
1,CECONY,"Sunrun Installation Services, Inc.",MC-473672,CENY-Q,Queens,11358.0,FL NWK,Corona_1,N,NaN,4.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NM,N,2/29/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,N
2,CECONY,"Sunrun Installation Services, Inc.",MC-473670,CENY-Q,Queens,11429.0,SD2738,Jamaica,N,NaN,2.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NM,N,2/29/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,N


In [3]:
# Keep only ESS, excluding thermal/ice
coned_df = coned_df[pd.notnull(coned_df['ESS\n(kWAC)'])]
coned_df = pd.DataFrame(coned_df.loc[coned_df['ICE\n(kWAC)'].isnull()])

# Drop if 'Division' = 'CENY-W'
coned_df = coned_df[coned_df.Division !='CENY-W']

# Keep columns
coned_df = pd.DataFrame(coned_df[[
    'Developer', 'Application / Job #', 'Division', 'Zip Code', 'Substation', 
    'PV\n(kWAC)', 'ESS\n(kWAC)', 'Metering\n(NA / NM / RNM / CDG)', 'Value Stack\n(Y/N)',
    'Start Date', 'Verification Testing or Final Acceptance Date', 'Final Letter of Acceptance Date', 'Project Complete (Y/N)'
]])

# Rename columns
coned_df = coned_df.rename(columns = {
    'Developer': 'developer',
    'Application / Job #': 'coned_id',
    'Division': 'borough',
    'Zip Code': 'zipcode',
    'Substation': 'substation',
    'PV\n(kWAC)': 'pv_kwac',
    'ESS\n(kWAC)': 'power_kwac',
    'Metering\n(NA / NM / RNM / CDG)': 'metering',
    'Value Stack\n(Y/N)': 'value_stack',
    'Start Date': 'application_date',
    'Verification Testing or Final Acceptance Date': 'verification_or_final_acceptance_date',
    'Final Letter of Acceptance Date': 'final_letter_of_acceptance_date',
    'Project Complete (Y/N)': 'project_complete'
})

# Rename boroughs
coned_df.loc[(coned_df.borough == 'CENY-BK'), 'borough'] = 'Brooklyn'
coned_df.loc[(coned_df.borough == 'CENY-BX'), 'borough'] = 'Bronx'
coned_df.loc[(coned_df.borough == 'CENY-M'), 'borough'] = 'Manhattan'
coned_df.loc[(coned_df.borough == 'CENY-Q'), 'borough'] = 'Queens'
coned_df.loc[(coned_df.borough == 'CENY-SI'), 'borough'] = 'Staten Island'

# Convert values to int
coned_df['zipcode'] = coned_df['zipcode'].astype(int)
coned_df['power_kwac'] = pd.to_numeric(coned_df['power_kwac'].astype(int))

# Convert to datetime
coned_df['application_date'] = pd.to_datetime(coned_df['application_date'])
coned_df['verification_or_final_acceptance_date'] = pd.to_datetime(coned_df['verification_or_final_acceptance_date'])
coned_df['final_letter_of_acceptance_date'] = pd.to_datetime(coned_df['final_letter_of_acceptance_date'])

coned_df.head(3)

,developer,coned_id,borough,zipcode,substation,pv_kwac,power_kwac,metering,value_stack,application_date,verification_or_final_acceptance_date,final_letter_of_acceptance_date,project_complete
67,"Raiden Electric, LLC",MC-473336,Brooklyn,10019,WATER_ST,NaN,2500,CDG,Y,2020-02-27,NaT,NaT,No
118,"Borrego Solar Systems, Inc.",MC-472871,Staten Island,10309,Woodrow,NaN,5000,CDG,Y,2020-02-25,NaT,NaT,No
206,Sunkeeper Solar,MC-472214,Staten Island,10007,Woodrow,NaN,5000,NaN,N,2020-02-20,NaT,NaT,No


In [4]:
# Datetime columns
coned_df['application_year'] = coned_df['application_date'].dt.strftime('%Y')
coned_df['application_month'] = coned_df['application_date'].dt.strftime('%m')
coned_df['application_ym'] = coned_df['application_date'].dt.strftime('%Y-%m')

coned_df['verification_year'] = coned_df['verification_or_final_acceptance_date'].dt.strftime('%Y')
coned_df['verification_month'] = coned_df['verification_or_final_acceptance_date'].dt.strftime('%m')
coned_df['verification_ym'] = coned_df['verification_or_final_acceptance_date'].dt.strftime('%Y-%m')

coned_df['acceptance_year'] = coned_df['final_letter_of_acceptance_date'].dt.strftime('%Y')
coned_df['acceptance_month'] = coned_df['final_letter_of_acceptance_date'].dt.strftime('%m')
coned_df['acceptance_ym'] = coned_df['final_letter_of_acceptance_date'].dt.strftime('%Y-%m')

coned_df.head(3)

,developer,coned_id,borough,zipcode,substation,pv_kwac,power_kwac,metering,value_stack,application_date,verification_or_final_acceptance_date,final_letter_of_acceptance_date,project_complete,application_year,application_month,application_ym,verification_year,verification_month,verification_ym,acceptance_year,acceptance_month,acceptance_ym
67,"Raiden Electric, LLC",MC-473336,Brooklyn,10019,WATER_ST,NaN,2500,CDG,Y,2020-02-27,NaT,NaT,No,2020,02,2020-02,NaN,NaN,NaN,NaN,NaN,NaN
118,"Borrego Solar Systems, Inc.",MC-472871,Staten Island,10309,Woodrow,NaN,5000,CDG,Y,2020-02-25,NaT,NaT,No,2020,02,2020-02,NaN,NaN,NaN,NaN,NaN,NaN
206,Sunkeeper Solar,MC-472214,Staten Island,10007,Woodrow,NaN,5000,NaN,N,2020-02-20,NaT,NaT,No,2020,02,2020-02,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Completed projects dataframe
coned_completed = coned_df[coned_df['project_complete'] == 'Yes']

# Applications dataframe
coned_apps = coned_df[coned_df['project_complete'] == 'No']

In [6]:
# Completed projects by year dataframe
coned_completed_ym = coned_completed.groupby('acceptance_ym').size().reset_index().rename(columns={0: 'completed'})
coned_completed_ym = coned_completed_ym.rename(columns={'acceptance_ym': 'date'})
#print('Completed Projects by Date:'), print(coned_completed_ym), print()

In [7]:
# Overall applications per year dataframe
coned_apps_total_ym = coned_df.groupby('application_ym').size().reset_index().rename(columns={0: 'apps_total'})
coned_apps_total_ym = coned_apps_total_ym.rename(columns={'application_ym': 'date'})
#print('Applications by Date (Overall History):'), print(coned_apps_total_ym), print()

In [8]:
# Pending applications dataframe
coned_apps_pending_ym = coned_apps.groupby('application_ym').size().reset_index().rename(columns={0: 'apps_pending'})
coned_apps_pending_ym = coned_apps_pending_ym.rename(columns={'application_ym': 'date'})
#print('Pending Applications by Date:'), print(coned_apps_pending_ym), print()

In [9]:
# Merge completed and overall dataframes
coned_projects_ym = pd.merge(coned_completed_ym, coned_apps_total_ym, on='date', how='outer')

In [10]:
# Merge coned_projects_ym and coned_apps_pending_ym
coned_projects_ym = pd.merge(coned_projects_ym, coned_apps_pending_ym, on='date', how='outer')

# Rearrange columns
coned_projects_ym = coned_projects_ym[['date', 'apps_total', 'apps_pending', 'completed']]

# Set year as index
coned_projects_ym.set_index('date', inplace=True)

# Write CSV
coned_projects_ym.to_csv('data/clean/coned_projects_ym.csv')
coned_projects_ym

,apps_total,apps_pending,completed
date,,,
2012-10,1.0,NaN,1.0
2013-06,NaN,NaN,1.0
2013-10,NaN,NaN,1.0
2014-02,NaN,NaN,1.0
2014-03,NaN,NaN,1.0
2015-04,NaN,NaN,1.0
2015-07,NaN,NaN,1.0
2016-08,NaN,NaN,4.0
2017-06,3.0,NaN,1.0
